# Class for Signal Data

In [1]:
import os
import re
import json
import numpy as np

from functionsMasterProjectMeinhart import get_sensor_data

In [2]:
def load_data_from_directory(file_dir, data_dict):
    '''
    Function to load all signal data from files of desired folder,
    then saving them to dictionary.
    '''
    # load all file names of desired folder
    files = []
    for (dirpath, dirnames, filenames) in os.walk(file_dir):
        files.extend(filenames)
        break
    
    # go through all files and save data to dictionary
    for ii in range(len(files)):
        
        current_rep_flag = False
        
        # check if file name contains subject, exercise and number of repetitions (e.g. 'subject01_RF_05.csv')
        if len(re.split('[_.]',files[ii])) is 4:
            # split current file
            current_sub = re.split('[_.]',files[ii])[0]
            current_ex  = re.split('[_.]',files[ii])[1]
            current_num_reps = re.split('[_.]',files[ii])[2]
            
        # check if file name contains additionaly current repetition (e.g. 'subject01_RF_05_01.csv')
        elif len(re.split('[_.]',files[ii])) is 5:
            current_rep_flag = True
            
            # split current file
            current_sub = re.split('[_.]',files[ii])[0]
            current_ex  = re.split('[_.]',files[ii])[1]
            current_num_reps = re.split('[_.]',files[ii])[2]
            current_rep = re.split('[_.]',files[ii])[3]
        
        # list for keys of dictionary (e.g. ['subject01','RS','10','02'])
        if current_rep_flag:
            keys = [current_sub, current_ex, current_num_reps, current_rep]
        else:
            keys = [current_sub, current_ex, current_num_reps]
        
        # create nested dictionary entry if it does not already exist
        if keys[0] not in data_dict:
            data_dict[keys[0]] = {}
        if keys[1] not in data_dict[keys[0]]:
            data_dict[keys[0]][keys[1]] = {}
        if keys[2] not in data_dict[keys[0]][keys[1]]:
            data_dict[keys[0]][keys[1]][keys[2]] = {}
        if current_rep_flag:
            if keys[3] not in data_dict[keys[0]][keys[1]][keys[2]]:
                data_dict[keys[0]][keys[1]][keys[2]][keys[3]] = {}
        
        # put current file path together
        file_path = os.path.join(file_dir, files[ii])
        
        # load signal data and save to dictionary
        if current_rep_flag:
            data_dict[keys[0]][keys[1]][keys[2]][keys[3]] = \
                get_sensor_data(in_file=file_path, signals=['Acc','Gyr','Mag'])
        else:
            data_dict[keys[0]][keys[1]][keys[2]] = \
                get_sensor_data(in_file=file_path, signals=['Acc','Gyr','Mag'])
        
                  

In [3]:
def print_dict_keys_func(data_dict, indent):
    
    indent += 1
    
    for key in [*data_dict]:
        print('\t' * indent + key)
            
        # check if dictionary is still nested
        if isinstance(data_dict[key], dict):
            print_dict_keys_func(data_dict[key], indent)
    
    indent -= 1
            

In [4]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
            np.int16, np.int32, np.int64, np.uint8,
            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, 
            np.float64)):
            return float(obj)
        elif isinstance(obj,(np.ndarray,)): #### This is the fix
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)


def save_dict_as_json_file(data_dict, save_path):
    # save dumped data to json file
    with open(save_path, 'w') as write_file:
        json.dump(data_dict, write_file, cls=NumpyEncoder, indent=2)

In [5]:
def convert_data_in_dictionary_to_numpyArray(data_dict):
    for key in [*data_dict]:
        # check if dictionary is still nested
        if isinstance(data_dict[key], dict):
            convert_data_in_dictionary_to_numpyArray(data_dict[key])
        else:
            data_dict[key] = np.array(data_dict[key])

Below is the actual class for the data

In [6]:
class SignalData():
    '''
    Class for signal data.
    '''
    def __init__(self, csv_dir=None, json_file=None, sampling_rate=256):
        self.csv_dir = csv_dir
        self.json_file = json_file
        self.sampling_rate = sampling_rate
        self.data_dict = {}
        
        # load data in the beginning if csv-directory or json-file name is given
        if csv_dir:
            load_data_from_directory(csv_dir, self.data_dict)
        elif json_file:
            self.get_data_json(json_file)
            
    def get_data_csv(self, csv_dir):
        self.data_dict.clear()
        load_data_from_directory(csv_dir, self.data_dict)
            
    def print_all_dict_keys(self):
        indent = -1
        if self.data_dict:
            print_dict_keys_func(self.data_dict, indent)
        else:
            print('No dictionary available.')
            
    def save_data_json(self, json_file_name, save_file_dir=None):
        if save_file_dir:
            json_file_path = os.path.join(save_file_dir, json_file_name)
        else:
            json_file_path = json_file_name
            
        save_dict_as_json_file(self.data_dict, json_file_path)
        
    def get_data_json(self, json_file_name, file_dir=None):
        if file_dir:
            json_file_path = os.path.join(file_dir, json_file_name)
        else:
            json_file_path = json_file_name
        
        self.data_dict.clear()
        with open(json_file_path, 'r') as read_file:
            self.data_dict = json.load(read_file)
    
        convert_data_in_dictionary_to_numpyArray(self.data_dict)
        

In [7]:
my_data = SignalData('E:\Physio_Data_Split_Exercise_test')

In [8]:
my_data.print_all_dict_keys()

subject01
	BC
		05
			Acc
			Gyr
			Mag
			time
		10
			Acc
			Gyr
			Mag
			time
		15
			Acc
			Gyr
			Mag
			time


In [9]:
my_data.data_dict['subject01']['BC']['05']['Acc']

array([[-0.8818359 ,  0.2978516 , -0.3535156 ],
       [-0.8740234 ,  0.3251953 , -0.3691406 ],
       [-0.8662109 ,  0.3447266 , -0.3969727 ],
       ...,
       [ 0.1010742 ,  0.4145508 ,  0.8261719 ],
       [ 0.09716797,  0.3681641 ,  0.8300781 ],
       [ 0.112793  ,  0.3525391 ,  0.8144531 ]])

In [8]:
my_data_2 = SignalData()
my_data_2.get_data_csv('E:\Physio_Data_Split_Exercise_test')

In [9]:
type(my_data_2.data_dict['subject01']['BC']['05']['Acc'])

numpy.ndarray

In [10]:
my_data.save_data_json('data_dict.json')

In [19]:
new_data = SignalData(json_file='data_dict.json')

In [21]:
type(new_data.data_dict['subject01']['BC']['05']['Acc'])

numpy.ndarray